In [0]:
%tensorflow_version 1.x

In [0]:
pip install numpy==1.16

For the code cell below you should see ‘Found GPU’ and tf version 1.x. Else remember to change runtime to GPU

In [0]:
!nvidia-smi

In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

Clome the Tensorflow API repo

In [0]:
!git clone https://github.com/tensorflow/models.git

Install some needed tools and dependencies.

In [0]:
%cd /content/models/research/
!protoc object_detection/protos/*.proto --python_out=.

In [0]:
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim'


Always run the cell below for every session restart

In [0]:
!python setup.py build
!python setup.py install

Check thr remaining time allowed for the session you are runing. Depending on your data try to know the number of hours that will be enough. If the time is lesser then restart the session. NB the time result you see in in hours

In [0]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', Left/3600)

Run Code below to see if all we need for the training has been installed.

In [0]:
#rember the last CD you did in order to specify the directory.
!pip install git+https://github.com/google-research/tf-slim.git
%cd /content/models/research/object_detection/builders/
!python model_builder_test.py

Change Directory to object detection folder

In [0]:
%cd /content/models/research/object_detection/

Get the pre-trained Object detection model from TensorFlow with the code below

In [0]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
!tar -xvf ssd_mobilenet_v2_coco_2018_03_29.tar.gz

In [0]:
pip install tensorboardcolab

Ensure to run the code cell below first

In [0]:
%load_ext tensorboard

IF you want to keep track of your model loss it best to run tensor board in background just like this before you run your training

In [0]:
%tensorboard --logdir training/

In [0]:
LOG_DIR = '/tmp/log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
with open ('ssd_mobilenet_v2_coco.config', 'w') as rsh:
    rsh.write('''\
    # Quantized trained SSD with Mobilenet v2 on Pascal VOC Dataset.
    model {
      ssd {
        num_classes: 15
        image_resizer {
          fixed_shape_resizer {
            height: 300
            width: 300
          }
        }
        feature_extractor {
          type: "ssd_mobilenet_v2"
          depth_multiplier: 1.0
          min_depth: 16
          conv_hyperparams {
            regularizer {
              l2_regularizer {
                weight: 3.99999989895e-05
              }
            }
            initializer {
              random_normal_initializer {
                mean: 0.0
                stddev: 0.00999999977648
              }
            }
            activation: RELU_6
            batch_norm {
              decay: 0.97000002861
              center: true
              scale: true
              epsilon: 0.0010000000475
            }
          }
          override_base_feature_extractor_hyperparams: true
        }
        box_coder {
          faster_rcnn_box_coder {
            y_scale: 10.0
            x_scale: 10.0
            height_scale: 5.0
            width_scale: 5.0
          }
        }
        matcher {
          argmax_matcher {
            matched_threshold: 0.5
            unmatched_threshold: 0.5
            ignore_thresholds: false
            negatives_lower_than_unmatched: true
            force_match_for_each_row: true
            use_matmul_gather: true
          }
        }
        similarity_calculator {
          iou_similarity {
          }
        }
        box_predictor {
          convolutional_box_predictor {
            conv_hyperparams {
              regularizer {
                l2_regularizer {
                  weight: 3.99999989895e-05
                }
              }
              initializer {
                random_normal_initializer {
                  mean: 0.0
                  stddev: 0.00999999977648
                }
              }
              activation: RELU_6
              batch_norm {
                decay: 0.97000002861
                center: true
                scale: true
                epsilon: 0.0010000000475
              }
            }
            min_depth: 0
            max_depth: 0
            num_layers_before_predictor: 0
            use_dropout: false
            dropout_keep_probability: 0.800000011921
            kernel_size: 1
            box_code_size: 4
            apply_sigmoid_to_scores: false
            class_prediction_bias_init: -4.59999990463
          }
        }
        anchor_generator {
          ssd_anchor_generator {
            num_layers: 6
            min_scale: 0.20000000298
            max_scale: 0.949999988079
            aspect_ratios: 1.0
            aspect_ratios: 2.0
            aspect_ratios: 0.5
            aspect_ratios: 3.0
            aspect_ratios: 0.333299994469
          }
        }
        post_processing {
          batch_non_max_suppression {
            score_threshold: 0.300000011921
            iou_threshold: 0.600000023842
            max_detections_per_class: 100
            max_total_detections: 100
          }
          score_converter: SIGMOID
        }
        normalize_loss_by_num_matches: true
        loss {
          localization_loss {
            weighted_smooth_l1 {
            }
          }
          classification_loss {
            weighted_sigmoid_focal {
              gamma: 2.0
              alpha: 0.75
            }
          }
          classification_weight: 1.0
          localization_weight: 1.0
        }
        encode_background_as_zeros: true
        normalize_loc_loss_by_codesize: true
        inplace_batchnorm_update: true
        freeze_batchnorm: false
      }
    }
    train_config {
      batch_size: 96
      data_augmentation_options {
        random_horizontal_flip {
        }
      }
      data_augmentation_options {
        ssd_random_crop {
        }
      }
      sync_replicas: true
      optimizer {
        adam_optimizer: {
          learning_rate: {
            manual_step_learning_rate {
              initial_learning_rate: .0002
              schedule {
                step: 500
                learning_rate: .00003
              }
              schedule {
                step: 1000
                learning_rate: .000002
              }
              schedule {
                step: 3000
                learning_rate: .0000003
              }
              schedule {
                step: 6000
                learning_rate: .00000003
              }
            }
          }
        }
        use_moving_average: false
      }
      fine_tune_checkpoint: "ssd_mobilenet_v2_coco_2018_03_29/model.ckpt"
      from_detection_checkpoint: true
      load_all_detection_checkpoint_vars: true
      num_steps: 50000
      startup_delay_steps: 0.0
      replicas_to_aggregate: 8
      max_number_of_boxes: 100
      unpad_groundtruth_tensors: false
      freeze_variables:
            [ 'FeatureExtractor/MobilenetV2/Conv/',
              'FeatureExtractor/MobilenetV2/expanded_conv/',
              'FeatureExtractor/MobilenetV2/expanded_conv_1/',
              'FeatureExtractor/MobilenetV2/expanded_conv_2/',
              'FeatureExtractor/MobilenetV2/expanded_conv_3/',
              'FeatureExtractor/MobilenetV2/expanded_conv_4/',
              'FeatureExtractor/MobilenetV2/expanded_conv_5/',
              'FeatureExtractor/MobilenetV2/expanded_conv_6/',
              'FeatureExtractor/MobilenetV2/expanded_conv_7/']
    }
    train_input_reader {
      label_map_path: "data/wildlife.v1.tfrecord/animals_label_map.pbtxt"
      tf_record_input_reader {
        input_path: "data/wildlife.v1.tfrecord/animals_train.tfrecord"
      }
    }
    eval_config: {
      num_examples: 750
      num_visualizations: 50
      eval_interval_secs: 0
    }
    eval_input_reader {
      label_map_path: "data/wildlife.v1.tfrecord/animals_label_map.pbtxt"
      shuffle: false
      num_readers: 1
      tf_record_input_reader {
        input_path: "data/wildlife.v1.tfrecord/animals_val.tfrecord"
      }
    }
    graph_rewriter {
      quantization {
        delay: 48000
        weight_bits: 8
        activation_bits: 8
      }
    }
    ''')

In [0]:
!wget https://github.com/HanYangZhao/tf_wildlife_detector/releases/download/1/wildlife.v1.tfrecord.zip
!unzip wildlife.v1.tfrecord.zip 'wildlife.v1.tfrecord/*' -d data

In [0]:
!python legacy/train.py --logtostderr --train_dir=training/ --pipeline_config_path=ssd_mobilenet_v2_coco.config

In [0]:
!python model_main.py \
  --pipeline_config_path=ssd_mobilenet_v2_coco.config\
  --model_dir=training/ \
  --num_train_steps=25000 \
  --num_eval_steps=200


**TfLite conversion**

In [0]:
!python3 export_tflite_ssd_graph.py \
  --pipeline_config_path="ssd_mobilenet_v2_coco.config" \
  --trained_checkpoint_prefix="training/model.ckpt-25000" \
  --output_directory="training" \
  --add_postprocessing_op=true

In [0]:
!tflite_convert \
  --output_file="training/output_tflite_graph.tflite" \
  --graph_def_file="training/tflite_graph.pb" \
  --inference_type=QUANTIZED_UINT8 \
  --input_arrays="normalized_input_image_tensor" \
  --output_arrays="TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3" \
  --mean_values=128 \
  --std_dev_values=128 \
  --input_shapes=1,300,300,3 \
  --change_concat_input_ranges=false \
  --allow_nudging_weights_to_use_fast_gemm_kernel=true \
  --allow_custom_ops

In [0]:
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!sudo apt-get update
!sudo apt-get install edgetpu-compiler

In [0]:
%cd /content/models/research/object_detection/training

In [0]:
!edgetpu_compiler output_tflite_graph.tflite

##EXporting inference graph

In [0]:
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path ssd_mobilenet_v2_coco.config --trained_checkpoint_prefix training/model.ckpt-1255 --output_directory trained_inference_graph/

##ZIPPing the EXported graph

In [0]:
!zip -r wildlife_graph.zip trained_inference_graph

#LETS TEST OUR MODEL AND SEE RESULT

> Copy about 9 images in the test_images folder and rename each to imagesX.jpg with X being a number from 1 to 9 before running the code below.



In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util



### Model preparation variable
MODEL_NAME = 'trained_inference_graph'
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = 'data/animals_label_map.pbtxt'
NUM_CLASSES = 15 #remember number of objects you are training? cool.


### Load a (frozen) Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')


###Loading label map
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)




### Load image into numpy function
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)




###STATING THE PATH TO IMAGES TO BE TESTED
PATH_TO_TEST_IMAGES_DIR = 'test_images/'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 4) ]
IMAGE_SIZE = (12, 8)




### Function to run inference on a single image which will later be used in an iteration
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: image})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.int64)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict



### To iterate on each image in the test image path defined 
### NB define the range of numbers and let it match the number of imAGES IN TEST FOLDER +1
for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=1)
  display(Image.fromarray(image_np))
  

